# Spotify Mood Based Song Retreiver

## 1. Data Extraction and Processing

In [176]:
# import requests as rq
# import bs4

In [177]:
# Extract data from the link - not being used currently
# def data_extractor(link):
#     print("Retrieving body text and title text from:", link)
#     res = rq.get(link)
#     soup = bs4.BeautifulSoup(res.text, 'lxml')
#     body = soup.find('body').text
#     title = soup.find('title').text
#     body = body.replace('\n', ' ')[:4000] #Change this to change the amount of text extracted
#     return body, title

In [178]:
# Defining the data extractor function
from langchain_community.document_loaders import WebBaseLoader
def data_extractor(link):
    loader = WebBaseLoader(link)
    docs = loader.load()
    return docs

In [179]:
# Define a sentiment analysis function
def sentiment(links):
    summ_data = []
    # Define the llm model and the chain
    llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro")
    chain = load_summarize_chain(llm, chain_type="stuff")
    # Extract data from each of the links and summarise it
    with open("summarised_text.txt", "w") as f:
        for link in links:
            docs = data_extractor(link)
            # Summarize the data and append it to the list
            result = chain.invoke(docs)
            f.write(result["output_text"])
            f.write("\n\n")
    with open("summarised_text.txt", "r") as s:
        with open("summariser_prompt.txt", "r") as f:
            prompt_template2 = f.read()
            prompt2 = PromptTemplate(input_variables=["input"], template=prompt_template2)
            chain2 = prompt2 | llm | StrOutputParser()
            return chain2.invoke(s.read())

## 2. Model Definition

In [180]:
# Get the API key from the .env file
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "True"

## 3. Primary Chain

In [181]:
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.docstore.document import Document


# Get the genres and moods according to the user browsing history

def song_retrieve(links):
    # Define a prompt
    with open("prompt.txt", "r") as f:
        prompt_template = f.read()
        prompt = PromptTemplate(input_variables=['data'], template=prompt_template)


    # Load the chain
    llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro")
    chain = prompt | llm | StrOutputParser()


    # Extract data from the link
    data = sentiment(links)


    # Run the chain
    result = chain.invoke(data)

    return result

In [182]:
links = [
    "https://www.youtube.com/watch?v=l6ev1lGq0B4",
    "https://www.reddit.com/r/AskReddit/comments/jdvj55/what_do_you_do_to_cheer_yourself_when_youre_sad/",
    "https://www.reddit.com/r/AskReddit/comments/3aosor/redditors_who_are_feeling_a_bit_sad_right_now/"
]

In [183]:
song_retrieve(links)

'Sad'